In [1]:
import torch
import torch.nn as nn

In [68]:
def lstm(bi, inpu_dim):
    inp_size, hidden_size, num_layers = inpu_dim, 1, 1
    return nn.LSTM(inp_size, hidden_size, num_layers, bidirectional=bi)
    

def printer(inpt, o, h):
    # [seq len, batch size, inp dimension]
    print('Input: \n')
    print(inpt)
    # [seq len, batch size, inp dimension * 2]
    print('Output: \n')
    print(o)
    # [num_layers * 2 , batch size, inp dimension]
    print('Hidden: \n')
    print(h)

In [69]:
inpt = torch.randn([4])
inpt = inpt.view(2, 2, 1)
model = lstm(False, inpu_dim = 1)

for name, param in model.named_parameters():
    param.data.fill_(1)
    
h = None

for t in inpt:
    if h == None:
        o, (h, c) = model(t.unsqueeze(0))
    else:
        o, (h, c) = model(t.unsqueeze(0), (h, c))
    printer(t, o, h)
    print('---------------------------------------')
    
model = lstm(False, inpu_dim = 1)

for name, param in model.named_parameters():
    param.data.fill_(1)

o, (h, c) = model(inpt)
printer(t, o, h)

Input: 

tensor([[ 0.9684],
        [-1.3935]])
Output: 

tensor([[[0.7019],
         [0.2180]]], grad_fn=<StackBackward>)
Hidden: 

tensor([[[0.7019],
         [0.2180]]], grad_fn=<StackBackward>)
---------------------------------------
Input: 

tensor([[-0.0419],
        [-0.2666]])
Output: 

tensor([[[0.8859],
         [0.7154]]], grad_fn=<StackBackward>)
Hidden: 

tensor([[[0.8859],
         [0.7154]]], grad_fn=<StackBackward>)
---------------------------------------
Input: 

tensor([[-0.0419],
        [-0.2666]])
Output: 

tensor([[[0.7019],
         [0.2180]],

        [[0.8859],
         [0.7154]]], grad_fn=<StackBackward>)
Hidden: 

tensor([[[0.8859],
         [0.7154]]], grad_fn=<StackBackward>)


In [28]:
inpt = torch.randn([8])
inpt = inpt.view(2, 2, 2) # Shape (inpt): [seq len, batch size, inp dimension]
print(inpt)
print("------------------------")

model = lstm(False, inpu_dim = 2)
for name, param in model.named_parameters():
    param.data.fill_(1)
h = None

inpx = torch.flip(inpt, [0, 1])


for t, tx in zip(inpt, inpx):
    if h == None:
        o, (h, c) = model(t.unsqueeze(0))
        ox, (hx, cx) = model(tx.unsqueeze(0))
    else:
        o, (h, c) = model(t.unsqueeze(0), (h, c))
        ox, (hx, cx) = model(tx.unsqueeze(0), (hx, cx))
    printer(t, o, h)
    print('--- Reverse')
    printer(tx, ox, hx)
    print("------------------------")
    
    
    
model = lstm(True, inpu_dim = 2)
for name, param in model.named_parameters():
    param.data.fill_(1)

o, (h, c) = model(inpt)
printer(t, o, h)

tensor([[[-0.1617,  0.7615],
         [ 0.8497, -1.3913]],

        [[ 1.5245,  0.6360],
         [-0.2786, -0.6527]]])
------------------------
Input: 

tensor([[-0.1617,  0.7615],
        [ 0.8497, -1.3913]])
Output: 

tensor([[[0.6760],
         [0.5045]]], grad_fn=<StackBackward>)
Hidden: 

tensor([[[0.6760],
         [0.5045]]], grad_fn=<StackBackward>)
--- Reverse
Input: 

tensor([[-0.2786, -0.6527],
        [ 1.5245,  0.6360]])
Output: 

tensor([[[0.3930],
         [0.7433]]], grad_fn=<StackBackward>)
Hidden: 

tensor([[[0.3930],
         [0.7433]]], grad_fn=<StackBackward>)
------------------------
Input: 

tensor([[ 1.5245,  0.6360],
        [-0.2786, -0.6527]])
Output: 

tensor([[[0.9492],
         [0.7264]]], grad_fn=<StackBackward>)
Hidden: 

tensor([[[0.9492],
         [0.7264]]], grad_fn=<StackBackward>)
--- Reverse
Input: 

tensor([[ 0.8497, -1.3913],
        [-0.1617,  0.7615]])
Output: 

tensor([[[0.7513],
         [0.9248]]], grad_fn=<StackBackward>)
Hidden: 

tensor(

In [32]:
h[-2,:,:] # forward RNN hidden state output 

tensor([[0.9492],
        [0.7264]], grad_fn=<SliceBackward>)

In [31]:
h[-1,:,:] # forward RNN hidden state output

tensor([[0.9248],
        [0.7513]], grad_fn=<SliceBackward>)

In [5]:
for name, param in lstm(False, inpu_size = 1).named_parameters():
    print(name)

weight_ih_l0
weight_hh_l0
bias_ih_l0
bias_hh_l0


In [6]:
for name, param in lstm(True, inpu_size = 1).named_parameters():
    print(name)

weight_ih_l0
weight_hh_l0
bias_ih_l0
bias_hh_l0
weight_ih_l0_reverse
weight_hh_l0_reverse
bias_ih_l0_reverse
bias_hh_l0_reverse


Basically, output of LSTM is the cummulative hidden states over all time steps.
While hidden state is over t time Step.
for Bi-directional, we get forward and backward both in output.
We can concatenate it or sum or multiply its just another hyper parameter and then pass it to linear layer to finally predict!